# BusSim experiments
    authors: Minh Kieu
    created: 21 Nov 2019
    version: 0.2 (jupyter)

#### Description: 

This code will apply Particle Filter on BusSim, requires:
1. model BusSim_stochastic.py and BusSim_deterministic.py
2. Calibration files in folder Calibration
3. Data files in Data folder 
4. Figures in Figures folder


#### Import Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from ParticleFilter_MK import ParticleFilter
from copy import deepcopy
import pandas as pd

#### Step 1: Generate synthetic data from BusSim-truth

Requires: BusSim_truth.py
Run BusSim-truth to generate the required data for analysis


In [ ]:
#parameters for the BusSim-truth models, change parameters here if needed
from BusSim_truth import Bus,BusStop,Model

NumberOfStop=20
minDemand=0.5
maxDemand=1
#Initialise the ArrivalRate and DepartureRate
ArrivalRate = np.random.uniform(minDemand / 60, maxDemand / 60, NumberOfStop)
DepartureRate = np.sort(np.random.uniform(0.05, 0.5,NumberOfStop))
#DepartureRate = np.linspace(0.05, 0.5,NumberOfStop)
DepartureRate[0]=0
TrafficSpeed=14
#Initialise the model parameters
model_params = {        
    "dt": 10,
    "minDemand":minDemand,
    "maxDemand":maxDemand,
    "NumberOfStop": NumberOfStop,
    "LengthBetweenStop": 2000,
    "EndTime": 6000,
    "Headway": 5 * 60,
    "BurnIn": 1 * 60,
    "AlightTime": 1,
    "BoardTime": 3,
    "StoppingTime": 3,
    "BusAcceleration": 3  # m/s          
}

In [ ]:
#plotting function (uses Matplotlib)
def plot(model_params, StateData, GroundTruth):
    # plotting the space-time diagram
    GroundTruth[GroundTruth == 0] = np.nan
    GroundTruth[GroundTruth > (model_params['NumberOfStop'] * model_params['LengthBetweenStop'])] = np.nan
    StateData[StateData == 0] = np.nan
    StateData[StateData > (model_params['NumberOfStop']*model_params['LengthBetweenStop'])] = np.nan
    plt.plot(StateData[:, range(1, np.size(GroundTruth, 1), 4)])    
    return

In [ ]:
#this is the function to run BusSim-truth
def run_model(model_params,TrafficSpeed,ArrivalRate,DepartureRate,IncreaseRate,do_ani,do_spacetime_plot,do_reps,uncalibrated):  #this function is to quickly run the model
    '''
    Model runing and plotting
    '''
    model = Model(model_params, TrafficSpeed,ArrivalRate,DepartureRate,IncreaseRate)

    if do_ani or do_spacetime_plot or uncalibrated:
        for time_step in range(int(model.EndTime / model.dt)):
            model.step()
            if do_ani:
                text0 = ['Traffic Speed:  %.2f m/s' % (model.TrafficSpeed)]
                plt.text(0, 0.025, "".join(text0))
                text2 = ['Alighted passengers:  %d' % (len(model.alighted_passengers))]
                plt.text(0, 0.02, "".join(text2))
                plt.text(0, 0.015, 'Number of waiting passengers')
                plt.plot(model.StopList, np.zeros_like(model.StopList), "|", markersize=20, alpha=0.3)
                plt.plot(model.StopList, np.zeros_like(model.StopList), alpha=0.3)
                for busstop in model.busstops:
                    # this is the value where you want the data to appear on the y-axis.
                    val = 0.
                    plt.plot(busstop.position, np.zeros_like(busstop.position) + val, 'ok', alpha=0.2)
                    plt.text(busstop.position, np.zeros_like(busstop.position) + val + 0.01, len(busstop.passengers))
                for bus in model.buses:
                    plt.plot(bus.position, np.zeros_like(bus.position) + val, '>', markersize=10)
                    text1 = ['BusID= %d, occupancy= %d, speed= %.2f m/s' % (bus.busID + 1, len(bus.passengers), bus.velocity)]
                    if bus.status != 0:
                        plt.text(0, -bus.busID * 0.003 - 0.01, "".join(text1))
                plt.axis('off')
                plt.pause(1 / 30)
                plt.clf()
            plt.show()
        if do_spacetime_plot :
            plt.figure(3, figsize=(16 / 2, 9 / 2))
            x = np.array([bus.trajectory for bus in model.buses]).T        
            t = np.arange(0, model.EndTime, model.dt)
            x[x <= 0 ] = np.nan
            x[x >= (model.NumberOfStop * model.LengthBetweenStop)] = np.nan            
            plt.ylabel('Distance (m)')
            plt.xlabel('Time (s)')
            plt.plot(t, x, linewidth=.5,linestyle = '-')
            plt.pause(1 / 300) 
        
        if  uncalibrated:
            plt.figure(3, figsize=(16 / 2, 9 / 2))
            #plt.clf()            
            x = np.array([bus.trajectory for bus in model.buses]).T        
            t = np.arange(0, model.EndTime, model.dt)
            x[x <= 0 ] = np.nan
            x[x >= (model.NumberOfStop * model.LengthBetweenStop)] = np.nan            
            plt.ylabel('Distance (m)')
            plt.xlabel('Time (s)')
            plt.plot(t, x, linewidth=.5,linestyle = '--')
            plt.pause(1 / 300)    
        
        '''
        Now export the output data
        '''
   
        GPSData = model.buses[0].trajectory
        for b in range(1, model.FleetSize):
            GPSData = np.vstack((GPSData, model.buses[b].trajectory))
        GPSData[GPSData < 0] = 0        
        GPSData=np.transpose(GPSData)

        StateData = model.buses[0].states
        for b in range(1, model.FleetSize):
            StateData = np.hstack((StateData, model.buses[b].states))
        StateData[StateData < 0] = 0
    
        import pandas as pd
        GroundTruth = pd.DataFrame(model.buses[0].groundtruth)
        for b in range(1, model.FleetSize):
            GroundTruth = np.hstack((GroundTruth, model.buses[b].groundtruth))
        GroundTruth[GroundTruth < 0] = 0
    
        ArrivalData = ()
        for b in range(len(model.busstops)):
            ArrivalData += tuple([model.busstops[b].arrival_time])
                
        return model, model_params, ArrivalData, StateData, GroundTruth, GPSData

    if do_reps: #make a lot of replications
        RepGPS = []
        
        for time_step in range(int(model.EndTime / model.dt)):
            model.step()                
        
        RepGPS = model.buses[0].trajectory
        for b in range(1, model.FleetSize):
            RepGPS = np.vstack((RepGPS, model.buses[b].trajectory))
        RepGPS[RepGPS < 0] = 0        
        RepGPS=np.transpose(RepGPS)       
      
        return RepGPS        

In [ ]:
#runing parameters    

do_reps = False #whether we should export the distribution of headways
do_spacetime_plot = False
do_ani = False
do_spacetime_rep_plot = False
uncalibrated=False
do_data_export_realtime = False
do_data_export_historical= True
do_two_plots = False
 

In [ ]:
#now we can generate data from BusSim-truth 
do_reps=True
NumReps = 20
GPSData = []
for IncreaseRate in range(1,20):
    print('Increase Rate = ',IncreaseRate)
    for r in range(NumReps):
        RepGPS = run_model(model_params,TrafficSpeed,ArrivalRate,DepartureRate,IncreaseRate,do_ani,do_spacetime_plot,do_reps,uncalibrated)             
        GPSData.append(RepGPS)            
    meanGPS = np.mean(GPSData,axis=0)
    stdGPS = np.std(GPSData,axis=0)
    name0 = ['./Data/Historical_data_IncreaseRate_',str(IncreaseRate),'.pkl']
    str1 = ''.join(name0)    
    with open(str1,'wb') as f2:
        pickle.dump([model_params,meanGPS,stdGPS],f2)

In [ ]:
#generate another set of synthetic real-time data as well

#this is the section where we export multiple pickles, each is for a synthetic 'real-time' GPS data of IncreaseRate
do_spacetime_plot = True        
for IncreaseRate in range(1,21,1):        
    model2 = Model(model_params, TrafficSpeed,ArrivalRate,DepartureRate,IncreaseRate)
    for time_step in range(int(model2.EndTime / model2.dt)):
        model2.step()

    #del model2.buses[0] #remove the first bus 

    plt.figure(3, figsize=(16 / 2, 9 / 2))
    plt.clf() 
    x = np.array([bus.trajectory for bus in model2.buses]).T        
    t = np.arange(0, model2.EndTime, model2.dt)
    x[x <= 0 ] = np.nan
    x[x >= (model2.NumberOfStop * model2.LengthBetweenStop)] = np.nan            
    plt.ylabel('Distance (m)')
    plt.xlabel('Time (s)')
    plt.plot(t, x, linewidth=1)
    #plt.pause(1 / 300) 
    #plt.plot([],[], 'r',linestyle = '-',linewidth=.5, label='Historical')
    #plt.plot([],[], 'black',linestyle = '-',linewidth=2, label='Real-time')
    #plt.legend()
    name0 = ['./Figures/Fig_spacetime_IncreaseRate_',str(IncreaseRate),'.pdf']
    str1 = ''.join(name0)
    plt.savefig(str1, dpi=200,bbox_inches='tight')    

    GroundTruth = pd.DataFrame(model2.buses[0].groundtruth)
    for b in range(1, model2.FleetSize):
        GroundTruth = np.hstack((GroundTruth, model2.buses[b].groundtruth))
    GroundTruth[GroundTruth < 0] = 0    

    name0 = ['./Data/Realtime_data_IncreaseRate_',str(IncreaseRate),'.pkl']
    str1 = ''.join(name0)    
    with open(str1,'wb') as f2:
        pickle.dump([model_params,t,x,GroundTruth],f2)


## Step 2: Doing nothing analysis (Base case scenario)

Requires: A02_doing_nothing_analysis.py

For this we want to test the prediction accuracy in real-time if we don't calibrate the models or doing any Data Assimilation.

To do this, you can use the console to run A02_doing_nothing_analysis.py, remember to change the working directory in the file first to run, or just run the below block

In [ ]:
import A02_doing_nothing_analysis
Results=A02_doing_nothing_analysis.IncreaseRate_analysis()

## Step 3: Calibration analysis

We now calibrate BusSim-deterministic and BusSim-stochastic against the synthetic data from BusSim-truth

### Step 3.1: Run BusSim_model_calibration.py 
Requires: BusSim_model_calibration.py

this Python function will implement Cross-Entropy Method to calibrate the BusSim models against the data from BusSim-truth
Remember to comment/uncomment the model you want to calibrate in the code, there are only two of them: BusSim-stochastic and BusSim-deterministic


In [ ]:
from BusSim_model_calibration import *
'''
Step 3.1.1: Model initiation and starting solution
'''   
CEM_params = { 
        'MaxIteration':50, #maximum number of iteration
        'NumSol': 500,  #number of solutions being evaluated at each iteration
        'NumRep': 15, #number of replication each time we run the model            
        'Elite': 0.10  #pick 15% of best solution for the next iteration
        }    
TrafficSpeed_std=1.5
TrafficSpeed_init=14

for maxDemand in range(1,10,2):
    maxDemand = maxDemand/2
    #load model parameters
    print('maxDemand  = ', maxDemand)
    model_params,meanGPS,stdGPS = unpickle_initiation(maxDemand)        

    #Starting solution   
    mean_arr = np.random.uniform(model_params['minDemand'] / 60, maxDemand/60, model_params['NumberOfStop']-2)
    std_arr = 0.025*np.ones(model_params['NumberOfStop']-2)
    mean_dep = np.sort(np.random.uniform(0.05, 0.5,model_params['NumberOfStop']-3))
    std_dep = 0.05*np.ones(model_params['NumberOfStop']-3)
    mean_traffic = np.random.uniform(TrafficSpeed_init - TrafficSpeed_std, TrafficSpeed_init + TrafficSpeed_std)
    std_traffic = TrafficSpeed_std
    '''
    Step 3.1.2: Main CEM loop for model calibration
    '''
    Sol_archived_mean = []
    Sol_archived_std = []
    best_PI=0
    PI_archived =[]
    last5best = np.zeros(5)
    for ite in range(CEM_params['MaxIteration']):
        '''
        Step 3.1.2.1: Generate solution
        '''
        print('Iteration number ',ite)        
        Sol_arr = np.zeros([CEM_params['NumSol'],1])
        for p in range(len(mean_arr)):            
            lower = 0             
            mu = mean_arr[p]
            sigma = std_arr[p]            
            upper = mu + 2*sigma
            temp_arr=truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=CEM_params['NumSol'])
            Sol_arr=np.append(Sol_arr,temp_arr[:,None],axis=1)
        Sol_arr = np.hstack([Sol_arr,np.zeros([CEM_params['NumSol'],1])])

        Sol_dep = np.zeros([CEM_params['NumSol'],2])
        for p in range(model_params['NumberOfStop']-3):            
            lower = 0
            mu = mean_dep[p]
            sigma = std_dep[p]            
            upper = mu + 2*sigma
            temp_arr=truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=CEM_params['NumSol'])
            Sol_dep=np.append(Sol_dep,temp_arr[:,None],axis=1)
        Sol_dep = np.hstack([Sol_dep,np.ones([CEM_params['NumSol'],1])])

        mu = mean_traffic
        sigma = std_traffic 
        lower = mu - 2*sigma
        upper = mu + 2*sigma
        Sol_traffic=truncnorm.rvs((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=CEM_params['NumSol'])

        #combine the solutions
        Sol0 = np.hstack([Sol_arr,Sol_dep,Sol_traffic[:,None]])    

        '''
        Step 3.1.2.2: Evaluate each solution in Sol0
        '''

        PIm = []
        for m in range(len(Sol0)):      
            #build up model
            SolGPS=[]
            for r in range(CEM_params['NumRep']):
                model = solution2model(model_params,Sol0[m])        
                RunGPS = run_model(model)             
                SolGPS.append(RunGPS)            
            meanSolGPS = np.mean(SolGPS,axis=0)
            stdSolGPS = np.std(SolGPS,axis=0)
            PIm.extend([np.mean(np.abs(meanSolGPS-meanGPS))+np.mean(np.abs(stdSolGPS-stdGPS))])
        Elist = np.argsort(PIm)[0:(int(CEM_params['Elite']*CEM_params['NumSol']))]
        print('Current best solution: ', PIm[Elist[0]]) #best solution

        '''
        Step 3.1.2.3: Generate new solutions for the next iteration
        '''
        mean_arr=np.mean(Sol_arr[Elist],axis=0)[1:model_params['NumberOfStop']-1]
        std_arr = np.std(Sol_arr[Elist],axis=0)[1:model_params['NumberOfStop']-1]
        mean_dep = np.mean(Sol_dep[Elist],axis=0)[2:model_params['NumberOfStop']-1]
        std_dep = np.std(Sol_dep[Elist],axis=0)[2:model_params['NumberOfStop']-1]
        mean_traffic = np.mean(Sol_traffic[Elist],axis=0)            
        std_traffic = np.std(Sol_traffic[Elist],axis=0)            

        '''
        Step 3.1.2.4: store the best solution and the current mean & std of the current generation of solutions
        '''
        if PIm[Elist[0]] > best_PI:
            best_PI = PIm[Elist[0]]
            best_mean = Sol0[Elist[0]]          
        sol_mean = np.concatenate([mean_arr,mean_dep,[mean_traffic]])
        sol_std = np.concatenate([std_arr,std_dep,[std_traffic]])
        Sol_archived_mean.append(sol_mean)
        Sol_archived_std.append(sol_std)
        PI_archived.append(PIm[Elist[0]])

        if abs((best_PI-np.mean(last5best))/best_PI) < 0.05 and abs((best_PI-last5best[-1])/best_PI) < 0.05: #break the loop if the best mean doesn't change more than 5% compared to the average best PI
            '''
            Step 3.1.3: Store the final parameter solution of BusSim                            '''

            name1 = ['./Calibration/BusSim_Model2_calibration_static_maxDemand_',str(maxDemand),'.pkl']
            str2 = ''.join(name1)
            with open(str2, 'wb') as f:
                pickle.dump([model_params, best_mean,Sol_archived_mean,Sol_archived_std,PI_archived], f)
            print('Output to file: ',str2)    
            break
        else: 
            last5best = np.concatenate([last5best[1:],[0]]) #otherwise keep storing the best_PI            
            last5best[-1]=best_PI



### Step 3.2: Analyse and plot the results (run A03_calibration.py)
Requires: A03_calibration.py

This file analyses the calibration results and plot the results after the calibration step

In [ ]:
import A03_calibration
Results = A03_calibration.IncreaseRate_analysis()

## Step 4: Calibration + Particle Filter
The objective here is to use BusSim to predict location of the buses at the next time steps

Requires: M01_BusSim_PF_v2.py and ParticleFilter_MK.py

Here we would call ParticleFilter_MK.py for the Particle Filter, M01_BusSim_PF_v2.py for the functions we develoepd to test Particle Filter


In [ ]:
from M01_BusSim_PF_v2 import *

In [ ]:
Results=IncreaseRate_analysis()